# Assignment-4:  Image classification using Convolutional Neural Networks (CNNs) to classify images. (Datasets: (Cats vs. Dogs) & (CIFAR-10)


## Anjali Kumari
25/AFI/17














## CATS VS DOGS DATASET- binary classification

In [11]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Conv2D, MaxPooling2D, Dense, Flatten,
    Dropout, BatchNormalization, Input,
    ReLU, LeakyReLU, Activation
)

from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from tensorflow.keras.initializers import HeNormal, GlorotUniform

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


In [12]:
import tensorflow_datasets as tfds

(ds_train, ds_test), ds_info = tfds.load(
    "cats_vs_dogs",
    split=["train[:80%]", "train[80%:]"],
    as_supervised=True,
    with_info=True
)


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/cats_vs_dogs/incomplete.P6TX6F_4.0.1/cats_vs_dogs-train.tfrecord*...:   0%…

Dataset cats_vs_dogs downloaded and prepared to /root/tensorflow_datasets/cats_vs_dogs/4.0.1. Subsequent calls will reuse this data.


/usr/local/lib/python3.12/dist-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


In [13]:
IMG_SIZE = 128
BATCH_SIZE = 32

def preprocess(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

ds_train = ds_train.map(preprocess).shuffle(1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
ds_test  = ds_test.map(preprocess).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)


In [14]:
def build_cnn_model(activation_fn, kernel_init):
    model = Sequential([
        Input(shape=(128, 128, 3)),

        Conv2D(32, (3,3), padding="same", kernel_initializer=kernel_init),
        BatchNormalization(),
        activation_fn(),
        MaxPooling2D((2,2)),

        Conv2D(64, (3,3), padding="same", kernel_initializer=kernel_init),
        BatchNormalization(),
        activation_fn(),
        MaxPooling2D((2,2)),

        Flatten(),
        Dense(64, kernel_initializer=kernel_init),
        activation_fn(),
        Dropout(0.4),

        Dense(1, activation="sigmoid")
    ])
    return model


In [15]:
experiments = [
    {
        "name": "ReLU + He + Adam",
        "activation": lambda: ReLU(),
        "initializer": HeNormal(),
        "optimizer": Adam(0.001)
    },
    {
        "name": "LeakyReLU + He + Adam",
        "activation": lambda: LeakyReLU(alpha=0.1),
        "initializer": HeNormal(),
        "optimizer": Adam(0.001)
    },

    {
        "name": "tanh + Glorot + Adam",
        "activation": lambda: Activation("tanh"),
        "initializer": GlorotUniform(),
        "optimizer": Adam(0.001)
    },
    {
        "name": "LeakyReLU + He + SGD",
        "activation": lambda: LeakyReLU(alpha=0.1),
        "initializer": HeNormal(),
        "optimizer": SGD(0.01, momentum=0.9)
    }
]


In [16]:
results_summary = {}
best_f1 = 0
best_model = None

for exp in experiments:
    print(f"\nTraining: {exp['name']}")

    model = build_cnn_model(exp["activation"], exp["initializer"])

    model.compile(
        optimizer=exp["optimizer"],
        loss="binary_crossentropy",
        metrics=["accuracy"]
    )

    model.fit(
        ds_train,
        validation_data=ds_test,
        epochs=5,
        verbose=1
    )

    # ---- Predictions ----
    y_true = []
    y_pred = []

    for images, labels in ds_test:
        preds = model.predict(images, verbose=0)
        preds = (preds > 0.5).astype(int).flatten()

        y_true.extend(labels.numpy())
        y_pred.extend(preds)

    # ---- Metrics ----
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred)
    rec = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)

    results_summary[exp["name"]] = {
        "Accuracy": acc,
        "Precision": prec,
        "Recall": rec,
        "F1-score": f1
    }

    print(f"Accuracy : {acc:.4f}")
    print(f"Precision: {prec:.4f}")
    print(f"Recall   : {rec:.4f}")
    print(f"F1-score : {f1:.4f}")

    if f1 > best_f1:
        best_f1 = f1
        best_model = exp["name"]

    tf.keras.backend.clear_session()



Training: ReLU + He + Adam
Epoch 1/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 102s 171ms/step - accuracy: 0.5898 - loss: 1.4310 - val_accuracy: 0.5570 - val_loss: 0.6512
Epoch 2/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 91s 154ms/step - accuracy: 0.6436 - loss: 0.6080 - val_accuracy: 0.7201 - val_loss: 0.5354
Epoch 3/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 90s 153ms/step - accuracy: 0.6830 - loss: 0.5722 - val_accuracy: 0.7506 - val_loss: 0.5089
Epoch 4/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 88s 149ms/step - accuracy: 0.7023 - loss: 0.5426 - val_accuracy: 0.7519 - val_loss: 0.5046
Epoch 5/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 144s 153ms/step - accuracy: 0.7213 - loss: 0.5353 - val_accuracy: 0.7683 - val_loss: 0.4737
Accuracy : 0.7683
Precision: 0.8153
Recall   : 0.7053
F1-score : 0.7563

Training: LeakyReLU + He + Adam
Epoch 1/5


/usr/local/lib/python3.12/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


582/582 ━━━━━━━━━━━━━━━━━━━━ 91s 155ms/step - accuracy: 0.5751 - loss: 3.6510 - val_accuracy: 0.6638 - val_loss: 0.6884
Epoch 2/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 91s 154ms/step - accuracy: 0.6735 - loss: 0.7953 - val_accuracy: 0.7405 - val_loss: 0.5512
Epoch 3/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 90s 152ms/step - accuracy: 0.7415 - loss: 0.5377 - val_accuracy: 0.7719 - val_loss: 0.4792
Epoch 4/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 91s 154ms/step - accuracy: 0.7803 - loss: 0.4598 - val_accuracy: 0.7835 - val_loss: 0.4590
Epoch 5/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 94s 159ms/step - accuracy: 0.8079 - loss: 0.4177 - val_accuracy: 0.7979 - val_loss: 0.4359


/usr/local/lib/python3.12/dist-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


Accuracy : 0.7979
Precision: 0.7887
Recall   : 0.8246
F1-score : 0.8063

Training: tanh + Glorot + Adam
Epoch 1/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 90s 151ms/step - accuracy: 0.5385 - loss: 0.9302 - val_accuracy: 0.5724 - val_loss: 0.6734
Epoch 2/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 90s 153ms/step - accuracy: 0.5742 - loss: 0.6773 - val_accuracy: 0.5505 - val_loss: 0.6779
Epoch 3/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 90s 154ms/step - accuracy: 0.6009 - loss: 0.6595 - val_accuracy: 0.5363 - val_loss: 0.7456
Epoch 4/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 91s 154ms/step - accuracy: 0.5998 - loss: 0.6574 - val_accuracy: 0.5484 - val_loss: 0.7638
Epoch 5/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 90s 154ms/step - accuracy: 0.6046 - loss: 0.6552 - val_accuracy: 0.5589 - val_loss: 0.7183


...

Accuracy : 0.5589
Precision: 0.8509
Recall   : 0.1636
F1-score : 0.2744

Training: LeakyReLU + He + SGD
Epoch 1/5


/usr/local/lib/python3.12/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


582/582 ━━━━━━━━━━━━━━━━━━━━ 69s 116ms/step - accuracy: 0.5584 - loss: 1.6113 - val_accuracy: 0.6851 - val_loss: 0.5873
Epoch 2/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 67s 114ms/step - accuracy: 0.6807 - loss: 0.5855 - val_accuracy: 0.7289 - val_loss: 0.5355
Epoch 3/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 68s 115ms/step - accuracy: 0.7320 - loss: 0.5284 - val_accuracy: 0.7371 - val_loss: 0.5424
Epoch 4/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 68s 115ms/step - accuracy: 0.7661 - loss: 0.4790 - val_accuracy: 0.7642 - val_loss: 0.4948
Epoch 5/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 69s 116ms/step - accuracy: 0.8008 - loss: 0.4319 - val_accuracy: 0.7276 - val_loss: 0.6000


/usr/local/lib/python3.12/dist-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


Accuracy : 0.7276
Precision: 0.7317
Recall   : 0.7357
F1-score : 0.7337


In [17]:
print("\nFINAL RESULTS")
for name, metrics in results_summary.items():
    print(f"\n{name}")
    for k, v in metrics.items():
        print(f"{k}: {v:.4f}")

print("\nBEST MODEL:")
print(best_model, "with F1 =", round(best_f1, 4))



FINAL RESULTS

ReLU + He + Adam
Accuracy: 0.7683
Precision: 0.8153
Recall: 0.7053
F1-score: 0.7563

LeakyReLU + He + Adam
Accuracy: 0.7979
Precision: 0.7887
Recall: 0.8246
F1-score: 0.8063

tanh + Glorot + Adam
Accuracy: 0.5589
Precision: 0.8509
Recall: 0.1636
F1-score: 0.2744

LeakyReLU + He + SGD
Accuracy: 0.7276
Precision: 0.7317
Recall: 0.7357
F1-score: 0.7337

BEST MODEL:
LeakyReLU + He + Adam with F1 = 0.8063


# CIFAR10 DATASET- multiclass classification

In [1]:
import tensorflow as tf
tf.config.run_functions_eagerly(True)

import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Conv2D, MaxPooling2D, Dense, Flatten,
    Dropout, BatchNormalization, Input, LeakyReLU
)
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from tensorflow.keras.initializers import (
    GlorotUniform, HeNormal, RandomNormal
)

In [2]:
# ===============================
# 1. LOAD & PREPROCESS DATA
# ===============================
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train = x_train.astype("float32") / 255.0
x_test  = x_test.astype("float32") / 255.0

y_train = y_train.squeeze()
y_test  = y_test.squeeze()

print("Train:", x_train.shape, y_train.shape)
print("Test :", x_test.shape, y_test.shape)


170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 8s 0us/step
Train: (50000, 32, 32, 3) (50000,)
Test : (10000, 32, 32, 3) (10000,)


In [3]:
def build_cnn_model(activation_fn, kernel_init):
    model = Sequential([
        Input(shape=(32, 32, 3)),

        Conv2D(32, (3,3), padding="same",
               kernel_initializer=kernel_init),
        BatchNormalization(),
        activation_fn(),          # ✅ CALL IT
        MaxPooling2D((2,2)),

        Conv2D(64, (3,3), padding="same",
               kernel_initializer=kernel_init),
        BatchNormalization(),
        activation_fn(),          # ✅ CALL IT
        MaxPooling2D((2,2)),

        Flatten(),
        Dense(64, kernel_initializer=kernel_init),
        activation_fn(),          # ✅ CALL IT
        Dropout(0.4),

        Dense(10, activation="softmax")
    ])
    return model


In [4]:
# Core
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

# Keras model & layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Conv2D, MaxPooling2D, Dense, Flatten,
    Dropout, BatchNormalization, Input,
    ReLU, LeakyReLU, Activation
)

# Dataset
from tensorflow.keras.datasets import cifar10

# Optimizers
from tensorflow.keras.optimizers import SGD, Adam, RMSprop

# Initializers
from tensorflow.keras.initializers import (
    HeNormal, GlorotUniform, RandomNormal
)


In [5]:
experiments = [
    {
        "name": "ReLU + He + Adam",
        "activation": lambda: tf.keras.layers.ReLU(),
        "initializer": HeNormal(),
        "optimizer": Adam(learning_rate=0.001)
    },
    {
        "name": "LeakyReLU + He + Adam",
        "activation": lambda: tf.keras.layers.LeakyReLU(alpha=0.1),
        "initializer": HeNormal(),
        "optimizer": Adam(learning_rate=0.001)
    },
    {
        "name": "ReLU + He + RMSprop",
        "activation": lambda: tf.keras.layers.ReLU(),
        "initializer": HeNormal(),
        "optimizer": RMSprop(learning_rate=0.001)
    },
    {
        "name": "tanh + Glorot + Adam",
        "activation": lambda: tf.keras.layers.Activation("tanh"),
        "initializer": GlorotUniform(),
        "optimizer": Adam(learning_rate=0.001)
    },

]


In [6]:
import numpy as np
import pandas as pd

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


In [7]:
metrics_results = []

for exp in experiments:
    print("\nEvaluating:", exp["name"])

    model = build_cnn_model(
        activation_fn=exp["activation"],
        kernel_init=exp["initializer"]
    )

    model.compile(
        optimizer=exp["optimizer"],
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )

    model.fit(
        x_train, y_train,
        epochs=5,
        batch_size=64,
        verbose=0
    )

    # ---- Predictions ----
    y_pred_prob = model.predict(x_test, verbose=0)
    y_pred = np.argmax(y_pred_prob, axis=1)

    # ---- Metrics ----
    acc = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average="weighted")
    recall = recall_score(y_test, y_pred, average="weighted")
    f1 = f1_score(y_test, y_pred, average="weighted")

    metrics_results.append({
        "Experiment": exp["name"],
        "Accuracy": acc,
        "Precision": precision,
        "Recall": recall,
        "F1-score": f1
    })

    tf.keras.backend.clear_session()



Evaluating: ReLU + He + Adam


/usr/local/lib/python3.12/dist-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(



Evaluating: LeakyReLU + He + Adam


/usr/local/lib/python3.12/dist-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(



Evaluating: ReLU + He + RMSprop


/usr/local/lib/python3.12/dist-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(



Evaluating: tanh + Glorot + Adam


/usr/local/lib/python3.12/dist-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


In [8]:
results_df = pd.DataFrame(metrics_results)
results_df = results_df.sort_values(by="Accuracy", ascending=False)
results_df


,Experiment,Accuracy,Precision,Recall,F1-score
1,LeakyReLU + He + Adam,0.7025,0.706069,0.7025,0.695493
2,ReLU + He + RMSprop,0.5476,0.570882,0.5476,0.532834
3,tanh + Glorot + Adam,0.5422,0.599965,0.5422,0.526477
0,ReLU + He + Adam,0.1000,0.010000,0.1000,0.018182


In [10]:
best_row = results_df.loc[results_df["F1-score"].idxmax()]

print("🏆 BEST MODEL FOUND 🏆")
print(best_row)


🏆 BEST MODEL FOUND 🏆
Experiment    LeakyReLU + He + Adam
Accuracy                     0.7025
Precision                  0.706069
Recall                       0.7025
F1-score                   0.695493
Name: 1, dtype: object
